<a href="https://colab.research.google.com/github/ilikemichael/ml1216/blob/main/embedding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#此處先下載一個 imdb情緒分析 檔案包,包含25000pos跟 25000neg 的語意資料
import tensorflow as tf

dataset = tf.keras.utils.get_file(
    fname="aclImdb.tar.gz", 
    origin="http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz", 
    extract=True,
)

84131840/84125825 [==============================] - 4s 0us/step


In [2]:
#下載的檔案存在dataset裡面
import glob
import os
base = os.path.dirname(dataset) #拿出前半段
glob.glob("/root/.keras/datasets/aclImdb/train/*")

['/root/.keras/datasets/aclImdb/train/urls_unsup.txt',
 '/root/.keras/datasets/aclImdb/train/pos',
 '/root/.keras/datasets/aclImdb/train/neg',
 '/root/.keras/datasets/aclImdb/train/unsupBow.feat',
 '/root/.keras/datasets/aclImdb/train/unsup',
 '/root/.keras/datasets/aclImdb/train/urls_neg.txt',
 '/root/.keras/datasets/aclImdb/train/labeledBow.feat',
 '/root/.keras/datasets/aclImdb/train/urls_pos.txt']

In [3]:
#下載的檔案存在dataset裡面, 利用glob.glob一層一層去看,裡面有pos, neg不同的語意, 此處作為說明筆記,下段才能執行程式
# import glob
# import os
# base = os.path.dirname(dataset) #拿出前半段
# glob.glob("/root/.keras/datasets/aclImdb/train/*")

In [4]:
import os
import glob
import pandas as pd

base = os.path.dirname(dataset)
def get_data(category):
    contents, targets = [], []
    dir = os.path.join(base, "aclImdb", category, "pos")
    lfn = glob.glob(os.path.join(dir, "*.txt"))
    ufn = glob.glob(os.path.join(dir, "*.TXT"))
    for fn in lfn + ufn:
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            targets.append(1)
    dir = os.path.join(base, "aclImdb", category, "neg")
    lfn = glob.glob(os.path.join(dir, "*.txt"))
    ufn = glob.glob(os.path.join(dir, "*.TXT"))
    for fn in lfn + ufn:
        with open(fn, "r", encoding="utf-8") as f:
            contents.append(f.read())
            targets.append(0)
    df = pd.DataFrame({
        "content":contents,
        "target":targets
    })
    return df
train_df = get_data("train")
test_df = get_data("test")

In [5]:
train_df

,content,target
0,Well... easily my favourite TV series ever. Ca...,1
1,This documentary makes you travel all around t...,1
2,"I think the film is educational. However, it f...",1
3,I just can't believe some of the comments on t...,1
4,Dig! I would say to anyone even if you don't l...,1
...,...,...
24995,"Those of you who, like me, were disappointed w...",0
24996,"""Little Man"", now on DVD, is a Wayans Brothers...",0
24997,Hollywood always had trouble coming to terms w...,0
24998,While William Shater can always make me smile ...,0


In [6]:
# MLP一定要先分詞,然後序列化,tokenize, sequence
# 這裡先Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer
tok = Tokenizer(num_words=3000)
# fit
tok.fit_on_texts(train_df["content"])

In [7]:
# transform
x_train_seq = tok.texts_to_sequences(train_df["content"])
x_test_seq = tok.texts_to_sequences(test_df["content"])

In [ ]:
pd.DataFrame(x_train_seq)
tok.index_word[3005]
tok.word_index

In [9]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train_pad = pad_sequences(x_train_seq, maxlen=512)
x_test_pad = pad_sequences(x_test_seq, maxlen=512)

In [10]:
pd.DataFrame(x_train_pad)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,499,5,567,69,35,29,10,67,132,6,12,11,6,41,1,61,120,364,16,12,10,241,499,5,103,117,2,117,171,36,377,5,1359,2,127,53,2951,9,57,50
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8,138,3,931,128,2974,463,1,272,31,578,8,2,72,2346,5,852,12,72,23,170,2,4,11,873,29,180,1189,11,659,1784,175,12,72,202,3,1138,2525,443,699
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,218,110,100,29,9,405,2616,80,295,194,20,2196,8,1,1114,437,33,78,46,20,11,1425,872,5,926,20,110,72,200,25,3,2211,46,248,100,29,1255,2161,15,2211
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,3,683,151,10,67,101,12,6,352,16,9,1,484,1,113,1,745,29,347,2497,1,422,225,6,1187,58,1217,296,76,3,2297,51,9,53,2,1,973,225,6,40
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,128,3,49,19,2,50,3,5,21,27,8,3,1139,71,12,14,3,608,2521,6,3,1165,1361,20,735,2,2,3,475,460,4,1,2,4,109,39,1780,5,27,801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24995,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,35,33,677,43,144,2,3,706,274,149,335,342,1341,892,22,63,444,1,201,2,354,43,36,464,126,2397,22,141,239,1771,1,1088,238,2194,311,600,467,43,4,155
24996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,136,1683,11,281,13,160,8,3,2386,69,37,93,18,33,2470,3,272,3,168,2,2017,94,46,160,136,1140,126,2355,20,44,22,178,46,144,916,131,501,7,7,2353
24997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,94,1,1323,50,5,794,187,463,91,1785,2571,14,3,676,39,1729,19,1,111,4,11,676,364,725,2699,408,12,77,25,54,28,86,1,102,77,847,213,5,1299,16
24998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,103,9,15,117,232,92,10,66,5,468,9,122,7,7,1,115,170,4,9,13,1018,1102,20,1,864,26,6,3,469,11,120,77,21,94,9,7,7,195,3,345


In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Dense, Dropout, Flatten
layers = [
    #Param# 就是3001*128 = 384128
    Embedding(3001, 128, mask_zero=True, input_length=512 ), #3001是因為還要加上0, output128是每一個詞都化成128個感受, 512個詞input,
    Flatten(),
    Dense(256, activation="relu"),
    Dropout(0.25),
    Dense(2, activation="softmax")
]
model = Sequential(layers)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 512, 128)          384128    
_________________________________________________________________
flatten (Flatten)            (None, 65536)             0         
_________________________________________________________________
dense (Dense)                (None, 256)               16777472  
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 514       
Total params: 17,162,114
Trainable params: 17,162,114
Non-trainable params: 0
_________________________________________________________________


In [12]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
model.compile(loss=SparseCategoricalCrossentropy(),
       optimizer="adam",
       metrics=["accuracy"])

In [13]:
y_train = train_df["target"]
y_test = test_df["target"]

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
callbacks = [
    EarlyStopping(patience=5, restore_best_weights=True),
    ModelCheckpoint("model.h5", save_best_only=True)
]
# validation_split: 切出一部分資料驗證
# batch_size: 看多少筆才做一次調整(梯度下降)
# epochs: 訓練次數(60000-6000筆/epoch)
# 1 epoch 多少次梯度下降: 54000 / 200 -> 270
# verbose: 印出多少log(1:default 0:quiet 2:)
model.fit(x_train_pad, 
     y_train,
     validation_split=0.1,
     batch_size=200,
     epochs=50,
     verbose=2,
     callbacks=callbacks)

Epoch 1/50
113/113 - 6s - loss: 0.5790 - accuracy: 0.7092 - val_loss: 0.3928 - val_accuracy: 0.8248
Epoch 2/50
113/113 - 4s - loss: 0.2374 - accuracy: 0.9059 - val_loss: 0.2172 - val_accuracy: 0.9220
Epoch 3/50
113/113 - 4s - loss: 0.1080 - accuracy: 0.9652 - val_loss: 0.3580 - val_accuracy: 0.8624
Epoch 4/50
113/113 - 4s - loss: 0.0296 - accuracy: 0.9950 - val_loss: 0.4882 - val_accuracy: 0.8408
Epoch 5/50
113/113 - 4s - loss: 0.0077 - accuracy: 0.9996 - val_loss: 0.4760 - val_accuracy: 0.8668
Epoch 6/50
113/113 - 4s - loss: 0.0029 - accuracy: 0.9999 - val_loss: 0.5409 - val_accuracy: 0.8548
Epoch 7/50
113/113 - 4s - loss: 0.0013 - accuracy: 1.0000 - val_loss: 0.5624 - val_accuracy: 0.8608


In [15]:
model.evaluate(x_test_pad, y_test)

782/782 [==============================] - 3s 4ms/step - loss: 0.3002 - accuracy: 0.8708


[0.30017349123954773, 0.8708400130271912]